# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [7]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [92]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Município'

r = requests.get(url)
r.status_code

200

In [93]:
data_json = r.json()
df_rj = pd.DataFrame(data_json['items'])

In [94]:
# 2) Quais são os três status das solicitações mais frequentes na base? Quais são suas frequências?
df_rj['status'].value_counts()


Deferido                                                 87
Arquivado                                                53
Regularizado                                             25
Arquivado por decurso de prazo                           23
Deferido (PVL-IF)                                        14
Encaminhado à PGFN com manifestação técnica favorável    11
Arquivado a pedido                                        4
Em retificação pelo interessado                           2
Em retificação pelo interessado (PVL-IF)                  1
Pendente de regularização                                 1
Processo pendente de distribuição                         1
Name: status, dtype: int64

In [68]:
# 3) Construa uma nova variável que contenha o ano do status. Observe que data_status vem como tipo object no DataFrame. 
#    Dica: você pode usar o método .str para transformar o tipo da variável em string, em seguida um método como slice() 
#    ou split().

df_rj['ano_status'] = df_rj['data_status'].str.slice(6,10)
df_rj.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status,ano_status
0,9308,Município,Nova Iguaçu,3303500,RJ,None,Regularizado,17944.000951/2007-14,2007-06-18T03:00:00Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Light - Serviços de Eletricidade S/A,Real,1451915.2,1,0,02/08/2007,2007
1,22824,Município,Rio de Janeiro,3304557,RJ,00000.000000/2032-87,Deferido,17944.000749/2016-75,2016-08-11T03:00:00Z,Operação contratual interna,PMAT,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,45240000.0,1,0,01/09/2016,2016


In [75]:
#4) Indique a frequência de cada ano do campo construído no item (3).
df_rj['ano_status'].value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2013    15
2020    15
2012    13
2011    10
2009     9
2015     9
2021     6
2017     5
2018     4
2006     4
2016     4
2002     4
2004     1
2003     1
Name: ano_status, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [98]:
#1) Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```)
#    , e que devolve os dados da consulta no formato *DataFrame*.

def consulta_pvl(estado: str, tpInteressado: str):
    
    url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf='+estado+'&tipo_interessado='+tpInteressado
    r = requests.get(url)
    data_json = r.json()
    df_pvl = pd.DataFrame(data_json['items'])
    return df_pvl


In [116]:
# 2) Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* 
#    em 'Arquivado por decurso de prazo' estão registradas?consulta_pvl('BA','Estado')

df_mg = consulta_pvl('MG','Estado')
df_mg['status'].value_counts()


Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado                                                 3
Arquivado pela STN                                        3
Arquivado por decurso de prazo                            1
Indeferido                                                1
Name: status, dtype: int64

In [134]:
# 3) Qual é o município da Bahia com mais solicitações deferidas?
df_ba = consulta_pvl('BA','Município') #consultando os registros dos municípios do estado da Bahia
df_ba_deferido = df_ba[(df_ba['status']=='Deferido')] #filtrando apenas os com status deferido
df_ba_deferido['interessado'].value_counts() #realizando a contagem por interessados

Luís Eduardo Magalhães    14
Vitória da Conquista      11
Lauro de Freitas          10
Camaçari                  10
Barreiras                  8
                          ..
Paulo Afonso               1
Ibotirama                  1
Nazaré                     1
Alcobaça                   1
Cruz Das Almas             1
Name: interessado, Length: 172, dtype: int64

In [143]:
# 4) Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'
df_ba_estado = consulta_pvl('BA','Estado') #consultado pvl do estado BA com interessado Estado
df_ba_estado.head(10)
df_ba_estado.to_csv('dataframe_BA_Estado.csv', index=False, sep=';', decimal=',', encoding='UTF-8')